In [17]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("WindowFunction").master("local[*]").getOrCreate()
spark

In [18]:


emp_data_1 = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"],
    ["021","101","John Smith","25","Female","50000","2016-02-15"],
]

emp_schema="employee_id string, department_id string,name string, age string,gender string,salary string, hire_date string"

In [19]:
emp_df=spark.createDataFrame(emp_data_1,emp_schema)
emp_df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [20]:
emp_unique=emp_df.distinct()
emp_unique.show()

[Stage 21:===========================================>              (6 + 2) / 8]

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|
|     

In [21]:
#select distinct department_id

emp_dept_ids=emp_df.select("department_id").distinct()
emp_dept_ids.show()

[Stage 24:===========================================>              (6 + 2) / 8]

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          103|
|          104|
|          105|
|          107|
|          106|
+-------------+



In [32]:
#window functions, min, max,sum,lead,lag, row_number,rank,dense_rank

from pyspark.sql.window import Window
from pyspark.sql.functions import max,col,desc,min

#select *, max(salary) over(partition by department_id order by salary desc) as max_salary_each_dept from tablenameabs


window_spec=Window.partitionBy("department_id").orderBy(col("salary").desc())
max_fun=max(col("salary")).over(window_spec)
min_fun=min(col("salary")).over(Window.partitionBy("department_id").orderBy(col("salary")))



dept_max_sal_df=emp_df.withColumn("max_salary",max_fun).withColumn("min_salary",min_fun)
dept_max_sal_df.show()

[Stage 42:==================================================>       (7 + 1) / 8]

+-----------+-------------+-------------+---+------+------+----------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|min_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+----------+
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|     45000|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|     45000|
|        021|          101|   John Smith| 25|Female| 50000|2016-02-15|     70000|     45000|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|     45000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|     55000|     48000|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|     55000|     48000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|     48000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|

In [34]:
# find the second highest salary in each department
from pyspark.sql.functions import dense_rank,col,desc
window_spec=Window.partitionBy("department_id").orderBy(col("salary").desc())
rank_fun=dense_rank().over(window_spec)

second_highest_sal=emp_df.withColumn("rank",rank_fun).filter("rank==2")
second_highest_sal.show()

[Stage 48:===========================================>              (6 + 2) / 8]

+-----------+-------------+-----------+---+------+------+----------+----+
|employee_id|department_id|       name|age|gender|salary| hire_date|rank|
+-----------+-------------+-----------+---+------+------+----------+----+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|   2|
|        021|          101| John Smith| 25|Female| 50000|2016-02-15|   2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|   2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|   2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|   2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|   2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|   2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|   2|
+-----------+-------------+-----------+---+------+------+----------+----+



In [27]:
from pyspark.sql.functions import expr
second_highest_using_sql=emp_df.withColumn("2nd_highest_sal",expr("dense_rank() over(partition by department_id order by salary desc)"))
second_highest_using_sql.where("2nd_highest_sal==2").show()

+-----------+-------------+-----------+---+------+------+----------+---------------+
|employee_id|department_id|       name|age|gender|salary| hire_date|2nd_highest_sal|
+-----------+-------------+-----------+---+------+------+----------+---------------+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|              2|
|        021|          101| John Smith| 25|Female| 50000|2016-02-15|              2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|              2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|              2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|              2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|              2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|              2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|              2|
+-----------+-------------+-----------+---+------+------+--------

In [ ]:
emp_df["bonus"]=emp_df["salary"]*2
emp_df.withColumn("bonus",col("salary")*2)